# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [2]:
### General parameters
max_steps = 5e10 ##5e5 # Set maximum number of steps to run environment.
run_path = "ppo" # The sub-directory name for model and summary statistics
load_model = False # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 10000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "ObstacleCurriculum" # Name of the training environment file.
curriculum_file = None

### Algorithm-specific parameters for tuning
gamma = 0.99 #0.99 # Reward discount rate.
lambd = 0.95 ##0.95 # Lambda parameter for GAE.
time_horizon = 2048 ##2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 ##1e-3 # Strength of entropy regularization
num_epoch = 5 ##5 # Number of gradient descent steps per batch of experiences.
num_layers = 2 ##2 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 ##0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 4096 ##2048 # How large the experience buffer should be before gradient descent.
learning_rate = 3e-4 # Model learning rate.
hidden_units = 128 ##64 # Number of units in hidden layer.
batch_size = 32 ##64 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [3]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'ObstacleCurriculumAcademy' started successfully!


Unity Academy name: ObstacleCurriculumAcademy
        Number of brains: 1
        Reset Parameters :
		
Unity brain name: ObstacleCurriculumBrain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 9
        Action space type: discrete
        Action space size (per agent): 8
        Memory space size (per agent): 0
        Action descriptions: , , , , , , , 


### Train the Agent(s)

In [4]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Step: 10000. Mean Reward: -1.4472185430463576. Std of Reward: 0.3865069569045623.
Step: 20000. Mean Reward: -1.4423856209150328. Std of Reward: 0.3932403034581145.
Step: 30000. Mean Reward: -1.648333333333333. Std of Reward: 0.521688591909104.
Step: 40000. Mean Reward: -1.908888888888887. Std of Reward: 0.8133887352108438.
Step: 50000. Mean Reward: -2.3485964912280672. Std of Reward: 0.9786034420692511.
Saved Model
Step: 60000. Mean Reward: -2.4949056603773547. Std of Reward: 1.0470085376406815.
Step: 70000. Mean Reward: -2.453725490196072. Std of Reward: 1.1260197309320443.
Step: 80000. Mean Reward: -2.4192307692307624. Std of Reward: 1.0911604552279368.
Step: 90000. Mean Reward: -2.3154629629629575. Std of Reward: 1.0781024433039161.
Step: 100000. Mean Reward: -2.1031147540983555. Std of Reward: 1.0953860626170775.
Saved Model
Step: 110000. Mean Reward: -2.787999999999987. Std of Reward: 1.2320084212374414.
Step: 120000. Mean Reward: -2.931911764705863. Std of Reward: 1.2956352148122

Step: 980000. Mean Reward: 0.5020312499999998. Std of Reward: 0.46549748148989745.
Step: 990000. Mean Reward: 0.482208588957055. Std of Reward: 0.47926326965877664.
Step: 1000000. Mean Reward: 0.4364242424242422. Std of Reward: 0.5483865607565209.
Saved Model
Step: 1010000. Mean Reward: 0.5337048192771082. Std of Reward: 0.4072664275248118.
Step: 1020000. Mean Reward: 0.5371893491124258. Std of Reward: 0.3951861964028943.
Step: 1030000. Mean Reward: 0.4903987730061349. Std of Reward: 0.4803213333919053.
Step: 1040000. Mean Reward: 0.5028612716763002. Std of Reward: 0.4778650054287421.
Step: 1050000. Mean Reward: 0.4640462427745663. Std of Reward: 0.5201362517696285.
Saved Model
Step: 1060000. Mean Reward: 0.493441176470588. Std of Reward: 0.46674411626629436.
Step: 1070000. Mean Reward: 0.45728571428571413. Std of Reward: 0.544092879226047.
Step: 1080000. Mean Reward: 0.523801169590643. Std of Reward: 0.43114582301164234.
Step: 1090000. Mean Reward: 0.4883035714285712. Std of Reward: 0

Step: 1940000. Mean Reward: 0.5913235294117646. Std of Reward: 0.38153580670666704.
Step: 1950000. Mean Reward: 0.5932035928143713. Std of Reward: 0.3539728420673288.
Saved Model
Step: 1960000. Mean Reward: 0.5675297619047619. Std of Reward: 0.4066599568961752.
Step: 1970000. Mean Reward: 0.5554705882352942. Std of Reward: 0.46690981843040175.
Step: 1980000. Mean Reward: 0.5307975460122701. Std of Reward: 0.584570243901726.
Step: 1990000. Mean Reward: 0.503814102564103. Std of Reward: 0.6472922179681582.
Step: 2000000. Mean Reward: 0.5168639053254439. Std of Reward: 0.5429420069384528.
Saved Model
Step: 2010000. Mean Reward: 0.5435542168674701. Std of Reward: 0.5673108357945625.
Step: 2020000. Mean Reward: 0.5450887573964497. Std of Reward: 0.4321309087529359.
Step: 2030000. Mean Reward: 0.5528012048192772. Std of Reward: 0.45178743028909163.
Step: 2040000. Mean Reward: 0.5688596491228068. Std of Reward: 0.29340362626247557.
Step: 2050000. Mean Reward: 0.6177556818181816. Std of Reward

Step: 2900000. Mean Reward: 0.5935838150289017. Std of Reward: 0.3830145715898863.
Saved Model
Step: 2910000. Mean Reward: 0.5348802395209582. Std of Reward: 0.6268462436693186.
Step: 2920000. Mean Reward: 0.5664912280701756. Std of Reward: 0.5271655129019978.
Step: 2930000. Mean Reward: 0.5677167630057803. Std of Reward: 0.4445977796534272.
Step: 2940000. Mean Reward: 0.6417403314917125. Std of Reward: 0.16811888709058007.
Step: 2950000. Mean Reward: 0.6252259887005648. Std of Reward: 0.11168562012482428.
Saved Model
Step: 2960000. Mean Reward: 0.6080898876404492. Std of Reward: 0.2181748932738219.
Step: 2970000. Mean Reward: 0.6108707865168538. Std of Reward: 0.21345591377167444.
Step: 2980000. Mean Reward: 0.6013999999999998. Std of Reward: 0.21751692216600402.
Step: 2990000. Mean Reward: 0.5996306818181817. Std of Reward: 0.22454880258074017.
Step: 3000000. Mean Reward: 0.5927049180327867. Std of Reward: 0.30509951852685896.
Saved Model
Step: 3010000. Mean Reward: 0.611229050279329

error: unpack requires a bytes object of length 4

### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [5]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/ppo\model-3200000.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo\model-3200000.cptk


INFO:tensorflow:Froze 4 variables.


INFO:tensorflow:Froze 4 variables.


Converted 4 variables to const ops.
